# Regresión Logística

Los modelos de regresión asumen que la variable de respuesta es cuantitativa, sin embargo, en muchas situaciones esta variable es cualitativa/categórica, por ejemplo el color de ojos. La idea de predecir variables categóricas es usualmente nombrada como _Clasificación_. Muchos de los problemas en los que se enfoca el Machine Learning están dentro de esta categoría, por lo mismo existen una serie de algoritmos y modelos con tal de obtener los mejores resultados. En esta clase introduciremos el algoritmo de clasificación más sencillo: _Regresión Logística_.

Recordemos que la **Regresión Lineal** considera un modelo de la siguiente forma:

$$ Y \approx X \theta $$

donde 
$$
Y = \begin{bmatrix}y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)}\end{bmatrix}
\qquad , \qquad
X = \begin{bmatrix} 
1 & x^{(1)}_1 & \dots & x^{(1)}_n \\ 
1 & x^{(2)}_1 & \dots & x^{(2)}_n \\
\vdots & \vdots & & \vdots \\
1 & x^{(m)}_1 & \dots & x^{(m)}_n
\end{bmatrix}
\qquad y \qquad
\theta = \begin{bmatrix}\theta_1 \\ \theta_2 \\ \vdots \\ \theta_m\end{bmatrix}
$$

y que se  entrenar una función lineal

$$h_{\theta}(x) = \theta_0 + \theta_1 x_1 + ... + \theta_n x_n$$

deforma que se minimice

$$J(\theta) = \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$


La **Regresión Logística** busca entrenar la función 

$$h_{\theta}(x) = \frac{1}{1 + e^{-(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n)}}$$

de forma que se minimice

$$J(\theta) = \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$

Es decir, el objetivo es encontrar un _"buen"_ vector $\theta$ de modo que

$$Y \approx g(X \theta)$$

en donde $g(z)$ es la función sigmoide (_sigmoid function_),

$$g(z) = \frac{1}{1+e^{-z}}$$

La función
sigmoide $g(z) = (1+e^{-z})^{-1}$ tiene la siguiente propiedad:

$$g'(z) =  g(z)(1-g(z))$$

"Demostración":

$$\begin{aligned}
g'(z) &= \frac{-1}{(1+e^{-z})^2} (-e^{-z}) \\
      &= \frac{e^{-z}}{(1+e^{-z})^2} \\
      &= \frac{1}{1+e^{-z}} \frac{e^{-z}}{1+e^{-z}} \\
      &= \frac{1}{1+e^{-z}} \left(1 - \frac{1}{1+e^{-z}} \right) \\
      &= g(z)(1-g(z))\end{aligned}$$

¡Es perfecta para encontrar un punto de corte y clasificar de manera binaria!

## Aproximación Ingenieril

¿Cómo podemos reutilizar lo que conocemos de regresión lineal?

Si buscamos minimizar
$$J(\theta) = \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$
Podemos calcular el gradiente y luego utilizar el método del máximo
descenso para obtener $\theta$.

El cálculo del gradiente es directo:

$$\begin{aligned}
\frac{\partial J(\theta)}{\partial \theta_k}
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) \frac{\partial}{\partial \theta_k} h_{\theta}(x^{(i)}) \\
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) \frac{\partial}{\partial \theta_k} g(\theta^T x^{(i)}) \\
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) h_{\theta}(x^{(i)}) \left(1-h_{\theta}(x^{(i)})\right) \frac{\partial}{\partial \theta_k} (\theta^T x^{(i)}) \\
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) h_{\theta}(x^{(i)}) \left(1-h_{\theta}(x^{(i)})\right) x^{(i)}_k\end{aligned}$$

¿Hay alguna forma de escribir todo esto de manera matricial? Recordemos
que si las componentes eran

$$\begin{aligned}
\sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) x^{(i)}_k = \sum_{i=1}^{m}  x^{(i)}_k \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)\end{aligned}$$

podíamos escribirlo vectorialmente como $$X^T (X\theta - Y)$$

Luego, para

$$\begin{aligned}
\frac{\partial J(\theta)}{\partial \theta_k}
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) h_{\theta}(x^{(i)}) \left(1-h_{\theta}(x^{(i)})\right) x^{(i)}_k \\
&= \sum_{i=1}^{m}  x^{(i)}_k \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) h_{\theta}(x^{(i)}) \left(1-h_{\theta}(x^{(i)})\right)\end{aligned}$$

podemos escribirlo vectorialmente como
$$\nabla_{\theta} J(\theta) = X^T  \Big[ (g(X\theta) - Y) \odot g(X\theta) \odot (1-g(X\theta)) \Big]$$
donde $\odot$ es la multiplicación elemento a elemento (element-wise) o producto Hadamard.

**Observación crucial:**
$$\nabla_{\theta} J(\theta) = X^T  \Big[ (g(X\theta) - Y) \odot g(X\theta) \odot (1-g(X\theta)) \Big]$$
no permite construir un sistema lineal para $\theta$, por lo cual sólo
podemos resolver iterativamente.

Por lo tanto tenemos el algoritmo

$$\begin{aligned}
\theta^{(n+1)} & = \theta^{(n)} - \alpha \nabla_{\theta} J(\theta^{(n)}) \\
\nabla_{\theta} J(\theta) &= X^T  \Big[ (g(X\theta) - Y) \odot g(X\theta) \odot (1-g(X\theta)) \Big]\end{aligned}$$



## Interpretación Probabilística

¿Es la derivación anterior
probabilísticamente correcta?

Asumamos que la pertenencia a los grupos está dado por

$$\begin{aligned}
\mathbb{P}[y = 1| \ x ; \theta ] & = h_\theta(x) \\
\mathbb{P}[y = 0| \ x ; \theta ] & = 1 - h_\theta(x)\end{aligned}$$

Esto es, una distribución de Bernoulli con $p=h_\theta(x)$.\
Las expresiones anteriores pueden escribirse de manera más compacta como

$$\begin{aligned}
\mathbb{P}[y | \ x ; \theta ] & = (h_\theta(x))^y (1 - h_\theta(x))^{(1-y)} \\\end{aligned}$$

La función de verosimilitud $L(\theta)$ nos
permite entender que tan probable es encontrar los datos observados,
para una elección del parámetro $\theta$.

$$\begin{aligned}
L(\theta) 
&= \prod_{i=1}^{m} \mathbb{P}[y^{(i)}| x^{(i)}; \theta ] \\
&= \prod_{i=1}^{m} \Big(h_{\theta}(x^{(i)})\Big)^{y^{(i)}} \Big(1 - h_\theta(x^{(i)})\Big)^{(1-y^{(i)})}\end{aligned}$$

Nos gustaría encontrar el parámetro $\theta$ que más probablemente haya
generado los datos observados, es decir, el parámetro $\theta$ que
maximiza la función de verosimilitud.

Calculamos la log-verosimilitud:

$$\begin{aligned}
l(\theta) 
&= \log L(\theta) \\
&= \log \prod_{i=1}^{m} (h_\theta(x^{(i)}))^{y^{(i)}} (1 - h_\theta(x^{(i)}))^{(1-y^{(i)})} \\
&= \sum_{i=1}^{m} y^{(i)}\log (h_\theta(x^{(i)})) + (1-y^{(i)}) \log (1 - h_\theta(x^{(i)}))\end{aligned}$$

No existe una fórmula cerrada que nos permita obtener el máximo de la
log-verosimitud. Pero podemos utilizar nuevamente el método del
gradiente máximo.

Recordemos que si

$$\begin{aligned}
g(z) = \frac{1}{1+e^{-z}}\end{aligned}$$

Entonces

$$\begin{aligned}
g'(z) &= g(z)(1-g(z))\end{aligned}$$

y luego tenemos que

$$\begin{aligned}
\frac{\partial}{\partial \theta_k} h_\theta(x) &= h_\theta(x) (1-h_\theta(x)) x_k\end{aligned}$$

$$\begin{aligned}
\frac{\partial}{\partial \theta_k} l(\theta) &=
\frac{\partial}{\partial \theta_k}  \sum_{i=1}^{m} y^{(i)}\log (h_\theta(x^{(i)})) + (1-y^{(i)}) \log (1 - h_\theta(x^{(i)})) \\
&= \sum_{i=1}^{m} y^{(i)}\frac{\partial}{\partial \theta_k}   \log (h_\theta(x^{(i)})) + (1-y^{(i)}) \frac{\partial}{\partial \theta_k}  \log (1 - h_\theta(x^{(i)})) \\
&= \sum_{i=1}^{m} y^{(i)}\frac{1}{h_\theta(x^{(i)})}\frac{\partial h_\theta(x^{(i)})}{\partial \theta_k} 
+ (1-y^{(i)}) \frac{1}{1 - h_\theta(x^{(i)})} \frac{\partial (1-h_\theta(x^{(i)}))}{\partial \theta_k} \\
&= \sum_{i=1}^{m} y^{(i)}(1-h_\theta(x^{(i)})) x^{(i)}- (1-y^{(i)}) h_\theta(x^{(i)}) x^{(i)}\\
&= \sum_{i=1}^{m} y^{(i)}x^{(i)}- y^{(i)}h_\theta(x^{(i)}) x^{(i)}- h_\theta(x^{(i)}) x^{(i)}+ y^{(i)}h_\theta(x^{(i)}) x^{(i)}\\
&= \sum_{i=1}^{m} (y^{(i)}-h_\theta(x^{(i)})) x^{(i)}\end{aligned}$$

Es decir, para maximizar la log-verosimilitud
obtenemos igual que para la regresión lineal:

$$\begin{aligned}
\theta^{(n+1)} & = \theta^{(n)} - \alpha \nabla_{\theta} l(\theta^{(n)}) \\
\frac{\partial l(\theta)}{\partial \theta_k}
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) x^{(i)}_k\end{aligned}$$

Aunque, en el caso de regresión logística, se tiene
$h_\theta(x)=1/(1+e^{-x^T\theta})$

## Hoy Regresión logistica binaria

Vamos a usar el conjunto de datos mtcars que viene preinstalado en R. Este conjunto de datos tiene varias variables, pero para simplificar, vamos a usar solo dos: mpg (millas por galón) y am (tipo de transmisión, 0 = automática, 1 = manual).

Primero, cargamos los datos y visualizamos las primeras filas:

In [1]:
# Cargar el conjunto de datos
data(mtcars)

# Ver las primeras filas del conjunto de datos
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [4]:
# Ajustar el modelo de regresión logística
modelo <- glm(am ~ mpg, family = binomial, data = mtcars)

# Resumen del modelo
summary(modelo)


Call:
glm(formula = am ~ mpg, family = binomial, data = mtcars)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)   
(Intercept)  -6.6035     2.3514  -2.808  0.00498 **
mpg           0.3070     0.1148   2.673  0.00751 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 43.230  on 31  degrees of freedom
Residual deviance: 29.675  on 30  degrees of freedom
AIC: 33.675

Number of Fisher Scoring iterations: 5


In [5]:
# Hacer predicciones con el modelo
predicciones <- predict(modelo, type = "response")

# Ver las primeras predicciones
head(predicciones)

Mazda RX4     Mazda RX4 Wag        Datsun 710    Hornet 4 Drive 
        0.4610951         0.4610951         0.5978984         0.4917199 
Hornet Sportabout           Valiant 
        0.2969009         0.2599331

In [7]:
# Convertir las probabilidades en predicciones de clase
predicciones_clase <- ifelse(predicciones > 0.5, 1, 0)

# Ver las primeras predicciones de clase
head(predicciones_clase)

Mazda RX4     Mazda RX4 Wag        Datsun 710    Hornet 4 Drive 
                0                 0                 1                 0 
Hornet Sportabout           Valiant 
                0                 0

In [9]:
data.frame(predicciones_clase,mtcars$am)

,predicciones_clase,mtcars.am
,<dbl>,<dbl>
Mazda RX4,0,1
Mazda RX4 Wag,0,1
Datsun 710,1,1
Hornet 4 Drive,0,0
Hornet Sportabout,0,0
Valiant,0,0
Duster 360,0,0
Merc 240D,1,0
Merc 230,1,0


# Comparación bivariada.

In [16]:
# Ajustar el modelo de regresión logística
modelo <- glm(am ~ wt, family = binomial, data = mtcars)

# Resumen del modelo
summary(modelo)

# Hacer predicciones con el modelo
predicciones <- predict(modelo, type = "response")

# Ver las primeras predicciones
head(predicciones)

# Convertir las probabilidades en predicciones de clase
predicciones_clase <- ifelse(predicciones > 0.5, 1, 0)

# Ver las primeras predicciones de clase
head(predicciones_clase)

data.frame(predicciones_clase,mtcars$am)


Call:
glm(formula = am ~ wt, family = binomial, data = mtcars)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)   
(Intercept)   12.040      4.510   2.670  0.00759 **
wt            -4.024      1.436  -2.801  0.00509 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 43.230  on 31  degrees of freedom
Residual deviance: 19.176  on 30  degrees of freedom
AIC: 23.176

Number of Fisher Scoring iterations: 6


Mazda RX4     Mazda RX4 Wag        Datsun 710    Hornet 4 Drive 
        0.8172115         0.6157283         0.9373069         0.2897304 
Hornet Sportabout           Valiant 
        0.1415972         0.1320944

Mazda RX4     Mazda RX4 Wag        Datsun 710    Hornet 4 Drive 
                1                 1                 1                 0 
Hornet Sportabout           Valiant 
                0                 0

,predicciones_clase,mtcars.am
,<dbl>,<dbl>
Mazda RX4,1,1
Mazda RX4 Wag,1,1
Datsun 710,1,1
Hornet 4 Drive,0,0
Hornet Sportabout,0,0
Valiant,0,0
Duster 360,0,0
Merc 240D,0,0
Merc 230,0,0


In [13]:
dim(mtcars)

[1] 32 11